# Instalar bibliotecas

In [1]:
!pip install cffi>=1.17.0 tqdm>=4.64.1 numpy<2.0.0;
!pip install pysus;

/bin/bash: line 1: 2.0.0: No such file or directory


In [60]:
!pip install xgboost

# Importar bibliotecas

In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
## models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from tqdm import tqdm

# Importar a Base de Dados

In [3]:
np.random.seed(42)

In [4]:
from pysus import SIH
sih = SIH().load()
sih.metadata

{'long_name': 'Sistema de Informações Hospitalares',
 'source': ('https://datasus.saude.gov.br/acesso-a-informacao/morbidade-hospitalar-do-sus-sih-sus/',
  'https://datasus.saude.gov.br/acesso-a-informacao/producao-hospitalar-sih-sus/'),
 'description': 'A finalidade do AIH (Sistema SIHSUS) é a de transcrever todos os atendimentos que provenientes de internações hospitalares que foram financiadas pelo SUS, e após o processamento, gerarem relatórios para os gestores que lhes possibilitem fazer os pagamentos dos estabelecimentos de saúde. Além disso, o nível Federal recebe mensalmente uma base de dados de todas as internações autorizadas (aprovadas ou não para pagamento) para que possam ser repassados às Secretarias de Saúde os valores de Produção de Média e Alta complexidade além dos valores de CNRAC, FAEC e de Hospitais Universitários – em suas variadas formas de contrato de gestão.'}

In [5]:
files = sih.get_files(["RD"], uf=["SP"], year=2024, month=[1, 2, 3])

In [6]:
sih.download(files)

RDSP2403.parquet: 100%|██████████| 849k/849k [00:53<00:00, 16.0kB/s]


[/root/pysus/RDSP2401.parquet,
 /root/pysus/RDSP2402.parquet,
 /root/pysus/RDSP2403.parquet]

In [7]:
for index in range(len(files)):
  parquet = sih.download(files[index])

17462541it [00:00, 15195685802.17it/s]
17268103it [00:00, 14367719398.00it/s]
18999157it [00:00, 25647969166.95it/s]


In [8]:
df = parquet.to_dataframe()

# Tratamento e transformação

In [14]:
df.fillna(0, inplace=True)

In [15]:
for col in df.columns:
    if pd.api.types.is_string_dtype(df[col]):
        stripped = df[col].dropna().str.strip()
        if stripped.apply(lambda x: x.replace('.', '', 1).isdigit()).all():
          df[col] = pd.to_numeric(stripped)

In [16]:
label_encoder = LabelEncoder()

for col in df:
  if df[col].dtype == 'string':
    print(col)
    df[col] = label_encoder.fit_transform(df[col])

CGC_HOSP
DIAG_PRINC
NUM_PROC
CPF_AUT
CID_NOTIF
GESTOR_DT
CNPJ_MANT
INFEHOSP
FAEC_TP
REMESSA
AUD_JUST
SIS_JUST
DIAGSEC1
DIAGSEC2
DIAGSEC3
DIAGSEC4
DIAGSEC5
DIAGSEC6
DIAGSEC7
DIAGSEC8
DIAGSEC9


In [17]:
df.head()

,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES,NASC,...,DIAGSEC9,TPDISEC1,TPDISEC2,TPDISEC3,TPDISEC4,TPDISEC5,TPDISEC6,TPDISEC7,TPDISEC8,TPDISEC9
0,350000,2024,3,3,0,3523103112078,1,13198260,353180,19670527,...,0,0,0,0,0,0,0,0,0,0
1,350000,2024,3,3,0,3523103112089,1,13058107,350950,19990607,...,0,1,0,0,0,0,0,0,0,0
2,350000,2024,3,3,0,3523103112090,1,13067104,350950,19620130,...,0,0,0,0,0,0,0,0,0,0
3,350000,2024,3,3,0,3523103112111,1,13476055,350160,20201016,...,0,1,0,0,0,0,0,0,0,0
4,350000,2024,3,3,0,3523103113508,1,13070260,350950,19760925,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df.shape

(241806, 113)

In [19]:
#df.to_csv('/content/drive/My Drive/Colab Notebooks/process.csv', index=False)

In [30]:
to_remove = [
    'UF_ZI', 'CEP', 'MUNIC_RES', 'NASC', 'SEXO', 'IDADE', 'COD_IDADE',
    'NACIONAL', 'RACA_COR', 'ETNIA', 'NUM_FILHOS', 'INSTRU', 'CBOR', 'CNAER',
    'VINCPREV', 'CONTRACEP1', 'CONTRACEP2', 'GESTRISCO', 'INSC_PN', 'IND_VDRL',
    'CID_NOTIF', 'HOMONIMO', 'VAL_SH', 'VAL_SP', 'VAL_SADT', 'VAL_RN',
    'VAL_ACOMP', 'VAL_ORTP', 'VAL_SANGUE', 'VAL_SADTSR', 'VAL_TRANSP',
    'VAL_OBSANG', 'VAL_PED1AC', 'VAL_UTI', 'US_TOT', 'VAL_UCI ', 'UTI_MES_AN',
    'VAL_SH_FED', 'VAL_SP_FED', 'VAL_SH_GES', 'VAL_SP_GES', 'UTI_MES_IN',
    'UTI_MES_AL', 'UTI_INT_IN', 'UTI_INT_AN', 'UTI_INT_AL', 'NATUREZA', 'NAT_JUR',
    'GESTAO', 'RUBRICA', 'MUNIC_MOV', 'AUD_JUST', 'SIS_JUST', 'MARCA_UCI',
    'NUM_PROC', 'TOT_PT_SP', 'CPF_AUT', 'SEQ_AIH5', 'GESTOR_COD', 'GESTOR_TP',
    'GESTOR_CPF', 'GESTOR_DT', 'CNPJ_MANT', 'INFESHOP', 'REGCT', 'SEQUENCIA',
    'REMESSA',  'DIAGSEC1', 'DIAGSEC2', 'TPDISEC7','TPDISEC8', 'TPDISEC9'
    'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 'DIAGSEC6', 'DIAGSEC7', 'DIAGSEC8',
    'DIAGSEC9', 'TPDISEC1', 'TPDISEC2', 'TPDISEC3', 'TPDISEC4', 'TPDISEC5',
    'TPDISEC6'
]

df = df.drop(columns=[col for col in to_remove if col in df.columns])
df.shape

(241806, 32)

# Preprocessamento

In [31]:
df_x = df.drop('VAL_TOT', axis=1)
df_y = df['VAL_TOT']

In [32]:
variace_threshold = 0.80

scaler = StandardScaler()
df_standardized = scaler.fit_transform(df_x)

pca = PCA()
pca.fit(df_standardized)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
pca_number = np.argmax(cumulative_variance >= 0.80) + 1
print(f"Número de componentes para 80% da variância: {pca_number}")

Número de componentes para 80% da variância: 13


In [33]:
pca = PCA(n_components=pca_number)
main_components = pca.fit_transform(df_standardized)
components_num = main_components.shape[1]
column_names = [f'PC{i}' for i in range(1, components_num + 1)]
pca_df = pd.DataFrame(data=main_components, columns=column_names)
pca_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13
0,-0.964252,-0.626439,-0.445290,0.788981,-0.555569,0.188585,-0.205014,0.347736,-0.537244,-0.108111,0.113023,-0.215031,-1.113443
1,-0.047968,0.760492,-0.770272,0.560271,-0.514258,-3.370100,0.135374,-1.487842,-0.489546,-2.735222,-3.977592,-2.327300,-2.113984
2,-3.215761,1.634052,-1.212236,0.405181,0.240161,1.735050,1.037307,0.603529,-0.903242,-0.177042,-0.034528,-0.358814,-0.891716
3,-0.969837,-0.602681,-0.555286,0.856715,-1.219990,-0.526281,0.176339,0.194222,0.088523,-0.079633,0.142237,-0.475414,-1.361853
4,-1.663163,0.085365,-0.726642,0.712213,-0.524780,0.385875,0.221311,0.403295,-0.461124,-0.108217,0.088418,-0.327143,-1.150521


In [35]:
#pca_df.to_csv('/content/drive/My Drive/Colab Notebooks/pca.csv', index=False)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(
    pca_df, df_y, test_size=0.3, random_state=42, shuffle=True
)

print(f"Dados -> Treinamento {len(x_train)} - Teste: {len(x_test)}")
print(f"Target -> Treinamento {len(y_train)} - Teste: {len(y_test)}")

Dados -> Treinamento 169264 - Teste: 72542
Target -> Treinamento 169264 - Teste: 72542


# Treinamento do Modelo

In [64]:
models = {
    "Random Forest": make_pipeline(
        RandomForestRegressor(n_estimators=200, max_depth=30, random_state=42, n_jobs=-1)
    ),
    "XGBoost": make_pipeline(
        XGBRegressor(n_estimators=200, max_depth=6, learning_rate=0.1,
                         objective='reg:squarederror', random_state=42, n_jobs=-1)
    ),
    "Neural Network": make_pipeline(
        MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000,
                     activation='relu', random_state=42)
    )
}

In [65]:
results = {}

In [67]:
for name in tqdm(models, desc="Treinando modelos"):
    model = models[name]
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[name] = {'MSE': mse, 'R2': r2}

Treinando modelos:   0%|          | 0/3 [02:18<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
df_resultados = pd.DataFrame(results).T
print(df_resultados.sort_values("R2", ascending=False))

In [ ]:
# Gráfico de barras comparando R² com rótulos e cor azul
plt.figure(figsize=(8, 5))
bars = plt.bar(df_resultados.index, df_resultados['R2'], color='blue')
plt.title("Comparação de R² entre Modelos")
plt.ylabel("R²")
plt.ylim(0, 1)
plt.grid(axis='y')
plt.xticks(rotation=15)

# Adicionar rótulos nas barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.01, f'{yval:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Gráfico de barras comparando MSE com rótulos e cor azul
plt.figure(figsize=(8, 5))
bars = plt.bar(df_resultados.index, df_resultados['MSE'], color='blue')
plt.title("Comparação de MSE entre Modelos")
plt.ylabel("MSE")
plt.grid(axis='y')
plt.xticks(rotation=15)

# Adicionar rótulos nas barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 1, f'{yval:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()